In [1]:
from torch import cuda
import torch
from torch import bfloat16
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
from colorama import Fore, Style

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


In [3]:
df = pd.read_csv('cricket_summary.csv', header=None)
model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

/home/nlp-08/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/nlp-08/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def LLM_OUTPUT(messages):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = model.generate(
        input_ids,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id)
    return outputs[0][input_ids.shape[-1]:]

In [5]:
for i, j in df.iterrows():
    # print(i)
    messages = [
        {"role": "system", "content": j[0] + "\n\n" + j[1] },
        {"role": "user", "content": 'As a sports journalist give textual summary of above match from data provided above in 1 paragraph .'}
    ]
    response = LLM_OUTPUT(messages)
    answer_text = tokenizer.decode(response, skip_special_tokens=True)
    print(Fore.RED + "Prompt : " + 'Analyse the match report and correct this phrase and summarize in 1 paragraph ?')
    print(Fore.GREEN + "Answer : " + answer_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 

In [ ]:
import time
import colorama
from colorama import Fore

colorama.init(autoreset=True)

for i, j in df.iterrows():
    # Allow user to change the prompt if needed
    custom_prompt = input("Enter the prompt (or press Enter to use default): ")
    
    if not custom_prompt:
        custom_prompt = 'Analyse the match report and correct this phrase and summarize in 1 paragraph ?'
    
    messages = [
        {"role": "system", "content": j[1]},
        {"role": "user", "content": custom_prompt}
    ]
    
    response = LLM_OUTPUT(messages)
    answer_text = tokenizer.decode(response, skip_special_tokens=True)
    
    print(Fore.RED + "Prompt: " + custom_prompt)
    print(Fore.GREEN + "Answer: " + answer_text)
    
    # Wait for the user to press Enter before continuing to the next iteration
    input(Fore.YELLOW + "\nPress Enter to continue to the next iteration...")


Enter the prompt (or press Enter to use default):  
